# Analysing Honorary Street Names in NYC

In [1]:
import pandas as pd

In [2]:
pwd

'/Users/anushasubramanian/Desktop/nyc_streetnames/data_analysis'

In [3]:
df = pd.read_csv("nyc_honorary_streetnames.csv")
df.head()

,Coordinates,Intro Number,Enactment Date,Enactment Year,Enactment Number,Category,Borough,New Name,Present Name,Membership,Biographical Information,Limits,Zip Code,Introduced by Council Member(s),Notes
0,MULTILINESTRING ((-73.97348929299994 40.789574...,Int. No. 480-A,08/14/2022,2022,2022/081,Street,Manhattan,Ballet Hispánico Way,West 89th Street,Group,This co-naming will commemorate the legacy of ...,Between Columbus Avenue and Amsterdam Avenue,10024,Brewer,NaN
1,MULTILINESTRING ((-73.97119481799996 40.792714...,Int. No. 825,10/10/2008,2008,2008/048,Street,Manhattan,James D. Garst Way,South side of West 94th Street,Male,James D. Garst was a longtime advocate for aff...,Between Columbus Avenue and Amsterdam Avenue,10025,Brewer,NaN
2,MULTILINESTRING ((-73.87603364599994 40.842055...,Int. No. 1103-A,12/28/2009,2009,2009/092,Street,Bronx,Supervising Fire Dispatcher Dennis Patrick O’ ...,East 180th Street,Male,Dennis Patrick O’ Connell was a lifelong resid...,Between Bronx Park Avenue and Devoe Avenue,10460,Rivera,NaN
3,MULTILINESTRING ((-73.78751479699997 40.773209...,Int. No. 1103-A,12/28/2009,2009,2009/092,Street,Queens,Ron Carey Avenue,28th Avenue,Male,Ron Carey was a highly regarded member of his ...,Between 203rd and 206th Streets,11360,Avella,NaN
4,MULTILINESTRING ((-73.96225602099997 40.663194...,Int. No. 1631,08/23/2019,2019,2019/158,Street,Brooklyn,Malbone Centennial Way,Empire Boulevard,Group,This co-naming would commemorate the Malbone S...,Between Flatbush Avenue and Washington Avenue,11225,Cumbo,NaN


In [5]:
# drop two that are not streets
df = df[df["Category"] != "Structure"]
len(df)

884

## Running some EDA

In [6]:
# categories and boroughs 
pd.crosstab(df['Membership'], df['Borough'], margins = True)

Borough,Bronx,Brooklyn,Manhattan,Queens,Staten Island,All
Membership,,,,,,
Female,39,57,50,43,7,196
Group,11,25,33,31,5,105
Male,99,171,101,154,27,552
Male/Female,2,4,2,3,1,12
Misc,2,1,3,13,0,19
All,153,258,189,244,40,884


In [2]:
136/884

0.15384615384615385

In [58]:
# rough counts are probably not great if not normalised 
# for example proportion of streets named after women out of all honorary named streets in brooklyn
pd.crosstab(df['Membership'], df['Borough'], normalize = 'columns', margins = True).map(lambda x: f"{x*100:.1f}%")

Borough,Bronx,Brooklyn,Manhattan,Queens,Staten Island,All
Membership,,,,,,
Female,25.5%,22.1%,26.5%,17.6%,17.5%,22.2%
Group,7.2%,9.7%,17.5%,12.7%,12.5%,11.9%
Male,64.7%,66.3%,53.4%,63.1%,67.5%,62.4%
Male/Female,1.3%,1.6%,1.1%,1.2%,2.5%,1.4%
Misc,1.3%,0.4%,1.6%,5.3%,0.0%,2.1%


In [ ]:
2.6, 2, 2.7, 6.5, 2.5

#### I want to save this in a csv and then add this to my mapping dataframe for datawrapper (in ```mapping_streetnames_nyc.ipynb```)

In [63]:
pd.crosstab(df['Membership'], df['Borough'], normalize = 'columns', margins = True).map(lambda x: f"{x*100:.1f}%").reset_index().to_csv("crosstab_borough.csv", index = False)

### roughly 3/5th of all co-named streets are named after men and only 1 in 5 streets are named after women in all of NYC.

Highest density of female named streets is found in Manhattan and Bronx 
Lowest density in Staten Island and Queens

In [11]:
pd.crosstab(df['Membership'], df['Category'], margins = True)

Category,Street,All
Membership,,
Female,196,196
Group,105,105
Male,552,552
Male/Female,12,12
Misc,19,19
All,884,884


In [29]:
pd.crosstab(df['Membership'], df['Enactment Year'], margins = True).drop(index = "All")

Enactment Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,All
Membership,,,,,,,,,,,,,,,,,,,,,
Female,6,2,10,12,13,14,9,10,7,6,...,9,13,10,4,1,2,9,7,17,196
Group,4,0,16,9,4,4,2,5,4,1,...,6,4,2,3,2,3,8,7,6,105
Male,19,9,36,24,30,36,16,36,33,21,...,19,28,15,23,7,7,26,27,41,552
Male/Female,0,0,1,0,2,0,0,1,0,0,...,1,1,0,1,0,0,1,2,0,12
Misc,0,0,5,1,0,0,0,1,6,0,...,0,0,2,0,0,0,0,1,0,19


In [48]:
membership_per_year = df.pivot_table(index = "Enactment Year",
              columns = "Membership",
               values = "New Name",
              aggfunc = "count").fillna(0).reset_index()

In [52]:
#ratio of men and women?

membership_per_year["mf_ratio"] = (membership_per_year["Female"] / membership_per_year ["Male"]).round(2)

In [57]:
membership_per_year.sort_values("mf_ratio", ascending = False)

Membership,Enactment Year,Female,Group,Male,Male/Female,Misc,mf_ratio
17,2018,10.0,2.0,15.0,0.0,2.0,0.67
6,2007,9.0,2.0,16.0,0.0,0.0,0.56
12,2013,10.0,7.0,19.0,1.0,0.0,0.53
3,2004,12.0,9.0,24.0,0.0,1.0,0.50
11,2012,10.0,1.0,21.0,0.0,0.0,0.48
15,2016,9.0,6.0,19.0,1.0,0.0,0.47
16,2017,13.0,4.0,28.0,1.0,0.0,0.46
4,2005,13.0,4.0,30.0,2.0,0.0,0.43
23,2024,17.0,6.0,41.0,0.0,0.0,0.41
5,2006,14.0,4.0,36.0,0.0,0.0,0.39
